## Data Science Capstone

### Week 5 Assignment

##### ----------------------------------------------------------------------------------------
### There are 3 Parts in the Notebook
#### Part 1 Capture Toronto data using BeautifulSoup and transfor into Dataframe. Line 1 -8
#### Part 2 Add Venues and Categories using Foursquare and Cluster Analysis. Line 9 - 14
#### Part 3 Employ Foursquare Query for Supermarket and Carpark List. Line
##### -----------------------------------------------------------------------------------------

#### Import all the libraries

In [1]:
# Download beautifulsoup4 library for webscraping, if not installed
# !conda install beautifulsoup4

from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: / 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2019.11.28-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::certifi-2019.11.28-py37_0
  - anaconda/osx-64::certifi-2019.11.28-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0
  - anaconda/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-

### Part 1

In [2]:
pd.set_option('max_colwidth', 400)

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [4]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighborhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighborhood']
df_pc.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
df_pc.shape

(180, 3)

##### Remove 'Not Assigned' and then Aggregate 

In [6]:
df_pcn = df_pc.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_pcn.columns = ['Postcode', 'Borough', 'Neighborhood']
df_pcn.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [7]:
df_pcn.shape

(180, 3)

### Total Boroughs and Neighbours

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_pcn['Borough'].unique()),
        df_pcn.shape[0]
    )
)

The dataframe has 11 boroughs and 180 neighborhoods.


In [9]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [10]:
df_pos = pd.merge(df_pcn, df_geo, on=['Postcode'], how='inner')

df_tor = df_pos[['Postcode','Borough', 'Neighborhood', 'Latitude', 'Longitude']].copy()

df_tor.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
downtownTor_data = df_tor[df_tor['Borough'] == 'Central Toronto'].reset_index(drop=True)
print(downtownTor_data.shape)
downtownTor_data.head()

(9, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [14]:
address = 'Central Toronto,Toronto,Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of CentralToronto using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtownTor_data['Latitude'], downtownTor_data['Longitude'], downtownTor_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

### Part 2

#### Explore Toronto Neighbourhoods

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH' # your Foursquare ID
CLIENT_SECRET = 'AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH
CLIENT_SECRET:AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2


In [17]:
downtownTor_data.loc[0, 'Neighborhood']

'Lawrence Park'

In [18]:
neighborhood_latitude = downtownTor_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtownTor_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtownTor_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [19]:
# type your answer here
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH&client_secret=AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2&v=20180605&ll=43.7280205,-79.3887901&radius=1000&limit=200'

In [20]:
results = requests.get(url).json()
#results

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Granite Club,Gym / Fitness Center,43.733043,-79.381986
2,Tim Hortons,Coffee Shop,43.727324,-79.379563
3,Granite Club President's Lounge,Café,43.733005,-79.382059
4,Glendon Bookstore,Bookstore,43.727024,-79.378976


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


### Explore Neighborhoods in Central Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [25]:
centrepark_venues = getNearbyVenues(downtownTor_data['Neighborhood'],
                                   latitudes=downtownTor_data['Latitude'],
                                   longitudes=downtownTor_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville


#### Let's check the size of the resulting dataframe

In [26]:
print(centrepark_venues.shape)
centrepark_venues.head()

(111, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot


#### Let's check how many venues were returned for each neighborhood

In [27]:
centrepark_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,8,8,8,8,8,8
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",1,1,1,1,1,1
"North Toronto West, Lawrence Park",19,19,19,19,19,19
Roselawn,4,4,4,4,4,4
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",17,17,17,17,17,17
"The Annex, North Midtown, Yorkville",20,20,20,20,20,20


In [28]:
print('There are {} uniques categories.'.format(len(centrepark_venues['Venue Category'].unique())))

There are 65 uniques categories.


### Analyze Each Neighborhood

In [29]:
# one hot encoding
centrepark_onehot = pd.get_dummies(centrepark_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centrepark_onehot['Neighborhood'] = centrepark_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [centrepark_onehot.columns[-1]] + list(centrepark_onehot.columns[:-1])
centrepark_onehot = centrepark_onehot[fixed_columns]

centrepark_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
centrepark_onehot.shape

(111, 66)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
centrepark_grouped = centrepark_onehot.groupby('Neighborhood').mean().reset_index()
centrepark_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.00,0.000000,0.000000,0.000,0.028571,0.00,0.000000,0.057143,...,0.000000,0.057143,0.000000,0.0,0.028571,0.028571,0.00,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.00,0.000000,0.000000,0.125,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.25,0.00,0.000000,0.000000
3,Lawrence Park,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.333333,0.000000,...,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000
5,"North Toronto West, Lawrence Park",0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.052632,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.052632
6,Roselawn,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000
7,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",0.058824,0.00,0.058824,0.058824,0.000,0.000000,0.00,0.000000,0.000000,...,0.058824,0.058824,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.058824,0.000000
8,"The Annex, North Midtown, Yorkville",0.000000,0.05,0.000000,0.000000,0.000,0.000000,0.05,0.000000,0.150000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.05,0.000000,0.000000


In [32]:
centrepark_grouped.shape

(9, 66)

### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in centrepark_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = centrepark_grouped[centrepark_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0     Pizza Place  0.09
1    Dessert Shop  0.09
2  Sandwich Place  0.09
3            Café  0.06
4             Gym  0.06


----Davisville North----
               venue  freq
0              Hotel  0.25
1               Park  0.12
2  Food & Drink Shop  0.12
3   Department Store  0.12
4                Gym  0.12


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2                 Park  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0          Swim School  0.33
1                 Park  0.33
2             Bus Line  0.33
3  American Restaurant  0.00
4  Rental Car Location  0.00


----Moore Park, Summerhill East----
                 venue  freq
0         Tennis Court   1.0
1  American Restaurant   0.0
2   Salon / Barbershop   0.0
3        Jewelry Store   0.0
4   Light Rail Station   0.0


----

### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = centrepark_grouped['Neighborhood']

for ind in np.arange(centrepark_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(centrepark_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pharmacy,Park
1,Davisville North,Hotel,Sandwich Place,Park,Breakfast Spot,Gym,Department Store,Food & Drink Shop,Garden,Flower Shop,Fried Chicken Joint
2,"Forest Hill North & West, Forest Hill Road Park",Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop
3,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station
4,"Moore Park, Summerhill East",Tennis Court,Yoga Studio,Gas Station,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

centrepark_grouped_clustering = centrepark_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(centrepark_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 2, 1, 3, 1, 1], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

centrepark_merged = downtownTor_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
centrepark_merged = centrepark_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

centrepark_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Yoga Studio,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Hotel,Sandwich Place,Park,Breakfast Spot,Gym,Department Store,Food & Drink Shop,Garden,Flower Shop,Fried Chicken Joint
2,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Coffee Shop,Clothing Store,Yoga Studio,Spa,Fast Food Restaurant,Metro Station,Mexican Restaurant,Park,Diner,Gym / Fitness Center
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Pizza Place,Sandwich Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pharmacy,Park
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Tennis Court,Yoga Studio,Gas Station,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(centrepark_merged['Latitude'], centrepark_merged['Longitude'], centrepark_merged['Neighborhood'], centrepark_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examine Clusters

##### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [39]:
centrepark_merged.loc[centrepark_merged['Cluster Labels'] == 0, centrepark_merged.columns[[1] + list(range(5, centrepark_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Bus Line,Park,Swim School,Yoga Studio,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station


#### Cluster 2

In [40]:
centrepark_merged.loc[centrepark_merged['Cluster Labels'] == 1, centrepark_merged.columns[[1] + list(range(5, centrepark_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Hotel,Sandwich Place,Park,Breakfast Spot,Gym,Department Store,Food & Drink Shop,Garden,Flower Shop,Fried Chicken Joint
2,Central Toronto,1,Coffee Shop,Clothing Store,Yoga Studio,Spa,Fast Food Restaurant,Metro Station,Mexican Restaurant,Park,Diner,Gym / Fitness Center
3,Central Toronto,1,Dessert Shop,Pizza Place,Sandwich Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pharmacy,Park
5,Central Toronto,1,Pub,Coffee Shop,American Restaurant,Sports Bar,Pizza Place,Liquor Store,Restaurant,Light Rail Station,Skating Rink,Fried Chicken Joint
8,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Indian Restaurant,Park,Pharmacy,Pizza Place,Pub,Liquor Store,Middle Eastern Restaurant


#### Cluster 3

In [41]:
centrepark_merged.loc[centrepark_merged['Cluster Labels'] == 2, centrepark_merged.columns[[1] + list(range(5, centrepark_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Tennis Court,Yoga Studio,Gas Station,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop


#### Cluster 4

In [42]:
centrepark_merged.loc[centrepark_merged['Cluster Labels'] == 3, centrepark_merged.columns[[1] + list(range(5, centrepark_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,3,Home Service,Garden,Music Venue,Donut Shop,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop


#### Cluster 5

In [43]:
centrepark_merged.loc[centrepark_merged['Cluster Labels'] == 0, centrepark_merged.columns[[1] + list(range(5, centrepark_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Bus Line,Park,Swim School,Yoga Studio,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station


## Cluster Analysis and interpretation

##### ------------------------------------------------------------------------------------------------------------------------------
#### Amont the total 5 clusters, Cluster 2 has the maximum number of Venues and Categories gives good choice and selection.
#### Line 33 Top 5 most common Venues are in Summer Hill West, The Annex and Davisille.
#### Line 35 Top 10 Venues with good Footfall are Davisville, Davisville North, Summer Hill Westand and The Annex.
#### Cluster Analysis based on the Postcode also confirn the above listed Neighbourhoods are best for setup Supermarket.
##### --------------------------------------------------------------------------------------------------------------------------------

### Part 3 Foursquare Query for Supermarket and Carpark data
#### Foursquare Query for generating list for Supermarket and Carpart in Central Toronto

## Super Markets list

In [44]:
CLIENT_ID = 'H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH' # your Foursquare ID
CLIENT_SECRET = 'AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH
CLIENT_SECRET:AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2


##### In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent foursquare_agent, as shown below.

In [45]:
address = 'Central Toronto,Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


#### Detup Foursquare Query

In [46]:
search_query = 'Super Market'
radius = 1000
print(search_query + ' .... OK!')

Super Market .... OK!


### Define the corresponding URL

In [47]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH&client_secret=AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2&ll=43.6534817,-79.3839347&v=20180604&query=Super Market&radius=1000&limit=30'

In [48]:
results = requests.get(url).json()
#results

### Get relevant part of JSON and transform it into a pandas dataframe

In [49]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(3)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood
0,4f23983ee4b0476578d9c52f,The Supermarket,"[{'id': '4bf58dd8d48988d116941735', 'name': 'Bar', 'pluralName': 'Bars', 'shortName': 'Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}, 'primary': True}]",v-1592479511,False,Augusta Ave,College,43.653247,-79.381865,"[{'label': 'display', 'lat': 43.65324668644528, 'lng': -79.38186505133272}]",168,CA,Toronto,ON,Canada,"[Augusta Ave (College), Toronto ON, Canada]",NaN,NaN
1,4c02b3bb0d0e0f47e181019a,Starbucks (Market at Longo's),"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",v-1592479511,False,NaN,NaN,43.653989,-79.384055,"[{'label': 'display', 'lat': 43.65398904116468, 'lng': -79.38405508244834}]",57,CA,NaN,NaN,Canada,[Canada],NaN,NaN
2,4c50d7d7250dd13a12fa377c,City of Toronto,"[{'id': '50aa9e094b90af0d42d5de0d', 'name': 'City', 'pluralName': 'Cities', 'shortName': 'City', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_', 'suffix': '.png'}, 'primary': True}]",v-1592479511,False,NaN,NaN,43.650072,-79.383888,NaN,379,CA,NaN,Ontario,Canada,"[Ontario, Canada]",NaN,NaN


In [50]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(3)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,The Supermarket,Bar,Augusta Ave,College,43.653247,-79.381865,"[{'label': 'display', 'lat': 43.65324668644528, 'lng': -79.38186505133272}]",168,CA,Toronto,ON,Canada,"[Augusta Ave (College), Toronto ON, Canada]",NaN,NaN,4f23983ee4b0476578d9c52f
1,Starbucks (Market at Longo's),Coffee Shop,NaN,NaN,43.653989,-79.384055,"[{'label': 'display', 'lat': 43.65398904116468, 'lng': -79.38405508244834}]",57,CA,NaN,NaN,Canada,[Canada],NaN,NaN,4c02b3bb0d0e0f47e181019a
2,City of Toronto,City,NaN,NaN,43.650072,-79.383888,NaN,379,CA,NaN,Ontario,Canada,"[Ontario, Canada]",NaN,NaN,4c50d7d7250dd13a12fa377c


#### Supermarket List and Map

In [51]:
dataframe_filtered.name

0                                   The Supermarket
1                     Starbucks (Market at Longo's)
2                                   City of Toronto
3                                    Farmers Market
4                               Superior Amusements
5                        Marcello's Market and Deli
6     Superior Plus Interlock and Construction Inc.
7            Mike's Independent City Market Toronto
8                   The Market by Longo's Elizabeth
9                                    The 888 Market
10                                      Super wraps
11                          Super Premium Ice Cream
12                            The Market by Longo’s
13                               Super Ordinary Lab
14                            The Market By Longo's
15                              Super Dollar & More
16                                       Super Stop
17             Superior Court of Justice Courthouse
18                                     Super Dollar
19          

In [52]:
Supermarket_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(Supermarket_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Supermarket_map)

# display map
Supermarket_map

### Part 3 Foursquare Query for Supermarket and Carpark data

### Carpark List


In [53]:
search_query = 'Car Park'
radius = 1000
print(search_query + ' .... OK!')

Car Park .... OK!


In [54]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=H3MVVRNSF40I5ZQ4YOPIP43NJMAKZAPJSFTOC5MTWEK0M0YH&client_secret=AXBX3WO1L5PQ322TCBCDX45MZUNS1VPLYU11KICDVZQ0IVN2&ll=43.6534817,-79.3839347&v=20180604&query=Car Park&radius=1000&limit=30'

In [55]:
results1 = requests.get(url).json()

In [56]:
# assign relevant part of JSON to venues
venues = results1['response']['venues']

# tranform venues into a dataframe
dataframe1 = json_normalize(venues)
dataframe1.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4cb747a1304576b07174b0c2,Carpark Parking,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'Parking', 'pluralName': 'Parking', 'shortName': 'Parking', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/parking_', 'suffix': '.png'}, 'primary': True}]",v-1592479512,False,211 Adelaide Street West,Simcoe St,43.648479,-79.386984,"[{'label': 'display', 'lat': 43.648479, 'lng': -79.386984}]",608,M5H 1X4,CA,Toronto,ON,Canada,"[211 Adelaide Street West (Simcoe St), Toronto ON M5H 1X4, Canada]",NaN
1,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}]",v-1592479512,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.652487552998565, 'lng': -79.39205307540142}]",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, Toronto ON, Canada]",NaN
2,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'Metro Station', 'pluralName': 'Metro Stations', 'shortName': 'Metro', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/subway_', 'suffix': '.png'}, 'primary': True}]",v-1592479512,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.660006357009195, 'lng': -79.39027220226043}]",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto ON M5G 2P1, Canada]",NaN
3,4daa07f5a86e771ea71ad28e,Green P Municipal Parking,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'Parking', 'pluralName': 'Parking', 'shortName': 'Parking', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/parking_', 'suffix': '.png'}, 'primary': True}]",v-1592479512,False,20 Dundas Square,Yonge St,43.656059,-79.379815,"[{'label': 'display', 'lat': 43.65605858456431, 'lng': -79.37981486320496}]",438,M5B 1N8,CA,Toronto,ON,Canada,"[20 Dundas Square (Yonge St), Toronto ON M5B 1N8, Canada]",NaN
4,5cecd749270ee7002c200608,Car Truck City,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'Auto Dealership', 'pluralName': 'Auto Dealerships', 'shortName': 'Auto Dealer', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/automotive_', 'suffix': '.png'}, 'primary': True}]",v-1592479512,False,2300 Yonge St,NaN,43.652197,-79.386634,"[{'label': 'display', 'lat': 43.652197212054595, 'lng': -79.3866340443492}]",260,M4P 1E4,CA,Toronto,ON,Canada,"[2300 Yonge St, Toronto ON M4P 1E4, Canada]",NaN


In [62]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns1]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1.head(3)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Carpark Parking,Parking,211 Adelaide Street West,Simcoe St,43.648479,-79.386984,"[{'label': 'display', 'lat': 43.648479, 'lng': -79.386984}]",608,M5H 1X4,CA,Toronto,ON,Canada,"[211 Adelaide Street West (Simcoe St), Toronto ON M5H 1X4, Canada]",4cb747a1304576b07174b0c2
1,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.652487552998565, 'lng': -79.39205307540142}]",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, Toronto ON, Canada]",4b54deadf964a520a3d027e3
2,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.660006357009195, 'lng': -79.39027220226043}]",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto ON M5G 2P1, Canada]",4b170e47f964a5208cc123e3


### Carpark list and Map

In [59]:
dataframe_filtered1.name

0                        Carpark Parking
1                            Grange Park
2            Queen's Park Subway Station
3              Green P Municipal Parking
4                         Car Truck City
5         Discount Car and Truck Rentals
6                    Trinity Square Park
7                       Lyft Car Service
8                      Larry Sefton Park
9     TTC Streetcar #403 - Victoria park
10                      Alamo Rent A Car
11                           Berczy Park
12                 Enterprise Rent-A-Car
13                         One Park Lane
14                       Avis Car Rental
15                 Enterprise Rent-A-Car
16      Residences at College Park North
17                     Osgoode Hall Park
18                 Enterprise Rent-A-Car
19                Physiomed College Park
20         Crunch Fitness - College Park
21                     Precise Park Link
22            Trinity Park And Labyrinth
23             Budget Car & Truck Rental
24      Toothwor

In [60]:
Carpark_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(Carpark_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Carpark_map)

# display map
Carpark_map

#### In Central Toronto there are 29 Supermarkets

##### Prepared by Dipak Jani